<a href='https://huggingface.co/learn/nlp-course/chapter7/7?fw=pt'> huggingface 1</a> </br>
<a href='https://huggingface.co/docs/transformers/v4.32.0/ko/tasks/question_answering'> huggingface 2</a>

# Preparing the data - SQuAD dataset

In [7]:
import torch
torch.__version__

'2.0.0+cu118'

In [8]:
from datasets import load_dataset

In [9]:
raw_datasets = load_dataset("squad")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [15]:
"""첫 번째 data 출력"""
print("* Context: ", raw_datasets["train"][0]["context"])
print("* Question: ", raw_datasets["train"][0]["question"])
print("* Answer: ", raw_datasets["train"][0]["answers"])
print("* Title: ", raw_datasets["train"][0]["title"])

* Context:  <class 'str'>
* Question:  To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
* Answer:  {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}
* Title:  University_of_Notre_Dame


In [11]:
"""
Context와 question field는 사용하기 편리하다. 반면에 answer field는 dic 형태여서 복잡하다.
Training 중에는, 가능한 답변이 하나뿐이므로 filter()를 사용해서 제거
즉, train dataset에서 정답(answer-text)이 복수 정답인 context가 있는지 확인.
"""
raw_datasets["train"].filter(lambda x: len(x["answers"]["text"]) != 1)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

In [12]:
"""이미 train dataset이 잘 구성되어 있기 때문에 수정하지 않아도 된다."""
len(raw_datasets['train'])

87599

In [13]:
"""
반면에, evaluation dataset은 가능한 답변이 여러 개 존재할 수 있다.
Validation dataset의 0, 2 번째 index 답 출력 
"""

print(raw_datasets["validation"][0]["answers"])
print(raw_datasets["validation"][2]["answers"])

{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'], 'answer_start': [177, 177, 177]}
{'text': ['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."], 'answer_start': [403, 355, 355]}


In [8]:
"""evaluation dataset의 경우, 깊게 다루지 않고 Datasets metric으로 간단하게 처리한다."""

'evaluation dataset의 경우, 깊게 다루지 않고 Datasets metric으로 간단하게 처리한다.'

# Training data preprocessing
질문의 답변에 대한 label을 생성하는 것이 어렵다. 이 레이블은 답변에 해당하는 토큰의 시작 및 끝 위치가 될 것이다. </br>
우선, 모델이 이해할 수 있는 ID로 Text를 변환해야 한다. 이를 위해서 <strong>Tokenizer</strong>를 사용.

In [9]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased" # distilbert-base-cased-distilled-squad
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
"""토큰화가 빠르게 구현되어 있는지 확인"""
tokenizer.is_fast

True

### BERT의 input은 [CLS] sentence_A [SEP] sentence_B [SEP] 이다.

In [11]:
"""Tokenizer로 check"""
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

"""[CLS] question [SEP] context [SEP]로 변형(id 형태로)"""
inputs = tokenizer(question, context)
"""ID 형태를 다시 원래 text로 decoding"""
tokenizer.decode(inputs["input_ids"])

'[CLS] to whom did the virgin mary allegedly appear in 1858 in lourdes france? [SEP] architecturally, the school has a catholic character. atop the main building\'s gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it, is a copper statue of christ with arms upraised with the legend " venite ad me omnes ". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian place of prayer and reflection. it is a replica of the grotto at lourdes, france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858. at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ), is a simple, modern stone statue of mary. [SEP]'

In [12]:
"""
*이 부분에서는 질문 응답 작업을 위한 라벨을 생성하는 방법과 훈련 데이터의 문맥이 
모델의 최대 길이(여기서는 384)를 초과할 수 있는 경우를 처리하는 방법을 설명하고 있습니다.

*질문 응답 모델의 목표는 주어진 텍스트 내에서 정답의 시작 및 끝 토큰의 인덱스를 예측하는 것입니다. 
따라서 각 토큰에 대한 시작 및 끝 위치 예측을 수행하도록 모델을 훈련해야 합니다.

*일부 예제의 문맥이 매우 길기 때문에 이를 처리하기 위해 슬라이딩 윈도우 방식을 사용합니다. 
현재 예제에서는 문맥의 최대 길이를 100으로 제한하고, 50개의 토큰을 가진 슬라이딩 윈도우를 사용하여 처리합니다. 
슬라이딩 윈도우 처리 방식은 문맥을 여러 개의 작은 조각으로 나누어 처리하는 것을 의미하며, 각 조각에 대한 예측을 수행합니다.

*또한 max_length를 사용하여 최대 길이를 설정하고, 
truncation을 "only_second"로 설정하여 문맥(두 번째 위치)이 질문과 함께 너무 길 경우 문맥을 자르고, 
stride를 사용하여 두 연속적인 조각 간의 겹치는 토큰 수를 설정하며, 
return_overflowing_tokens를 True로 설정하여 넘친 토큰을 유지하도록 tokenizer에 알립니다.
"""

'\n*이 부분에서는 질문 응답 작업을 위한 라벨을 생성하는 방법과 훈련 데이터의 문맥이 \n모델의 최대 길이(여기서는 384)를 초과할 수 있는 경우를 처리하는 방법을 설명하고 있습니다.\n\n*질문 응답 모델의 목표는 주어진 텍스트 내에서 정답의 시작 및 끝 토큰의 인덱스를 예측하는 것입니다. \n따라서 각 토큰에 대한 시작 및 끝 위치 예측을 수행하도록 모델을 훈련해야 합니다.\n\n*일부 예제의 문맥이 매우 길기 때문에 이를 처리하기 위해 슬라이딩 윈도우 방식을 사용합니다. \n현재 예제에서는 문맥의 최대 길이를 100으로 제한하고, 50개의 토큰을 가진 슬라이딩 윈도우를 사용하여 처리합니다. \n슬라이딩 윈도우 처리 방식은 문맥을 여러 개의 작은 조각으로 나누어 처리하는 것을 의미하며, 각 조각에 대한 예측을 수행합니다.\n\n*또한 max_length를 사용하여 최대 길이를 설정하고, \ntruncation을 "only_second"로 설정하여 문맥(두 번째 위치)이 질문과 함께 너무 길 경우 문맥을 자르고, \nstride를 사용하여 두 연속적인 조각 간의 겹치는 토큰 수를 설정하며, \nreturn_overflowing_tokens를 True로 설정하여 넘친 토큰을 유지하도록 tokenizer에 알립니다.\n'

In [13]:
"""Train dataset의 zero index data(question, context)를 tokenizer에게 전달 -> ID 형태로 변환"""
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
)

"""ID를 다시 text로 변환해서 출력"""
for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] to whom did the virgin mary allegedly appear in 1858 in lourdes france? [SEP] architecturally, the school has a catholic character. atop the main building's gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it, is a copper statue of christ with arms upraised with the legend " venite ad me omnes ". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the gr [SEP]
[CLS] to whom did the virgin mary allegedly appear in 1858 in lourdes france? [SEP] main building and facing it, is a copper statue of christ with arms upraised with the legend " venite ad me omnes ". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian place of prayer and reflection. it is a replica of the grotto at lourdes, france where the virgin mary reputedly appeared to saint [SEP]
[CLS] to whom did the virgin mary allegedly appear in 1858 in lourdes franc

In [14]:
"""
*이 예제는 네 개의 입력으로 분할되었습니다. 
각 입력은 질문과 일부 문맥을 포함하고 있습니다. 
질문에 대한 답변 ("Bernadette Soubirous")은 세 번째와 마지막 입력에서만 나타나므로 이렇게 긴 문맥을 처리하면 
답변이 문맥에 포함되지 않은 훈련 예제가 생성됩니다. 
이러한 경우 레이블은 start_position = end_position = 0으로 설정됩니다([CLS] 토큰을 예측하게 됩니다). 
또한, 답변이 잘리면(시작 또는 끝 부분만 있는 경우) 해당 레이블도 0으로 설정됩니다. 
답변이 완전히 문맥에 포함된 경우 레이블은 답변이 시작하는 토큰과 답변이 끝나는 토큰의 인덱스가 됩니다.

*데이터셋은 답변의 시작 위치를 문맥에서 제공하며, 답변의 길이를 추가함으로써 답변의 끝 위치를 찾을 수 있습니다. 
이러한 위치를 토큰 인덱스로 매핑하려면 offset 매핑을 사용해야 합니다. 
return_offsets_mapping=True를 사용하여 tokenizer로부터 이러한 offset 매핑을 얻을 수 있습니다.

*즉, answer span이 context에 같이 존재하지 않으면 (0,0)의 index를 반환한다.
또한, 같은 context임을 알려주기 위해 offset을 도입.
"""

'\n*이 예제는 네 개의 입력으로 분할되었습니다. \n각 입력은 질문과 일부 문맥을 포함하고 있습니다. \n질문에 대한 답변 ("Bernadette Soubirous")은 세 번째와 마지막 입력에서만 나타나므로 이렇게 긴 문맥을 처리하면 \n답변이 문맥에 포함되지 않은 훈련 예제가 생성됩니다. \n이러한 경우 레이블은 start_position = end_position = 0으로 설정됩니다([CLS] 토큰을 예측하게 됩니다). \n또한, 답변이 잘리면(시작 또는 끝 부분만 있는 경우) 해당 레이블도 0으로 설정됩니다. \n답변이 완전히 문맥에 포함된 경우 레이블은 답변이 시작하는 토큰과 답변이 끝나는 토큰의 인덱스가 됩니다.\n\n*데이터셋은 답변의 시작 위치를 문맥에서 제공하며, 답변의 길이를 추가함으로써 답변의 끝 위치를 찾을 수 있습니다. \n이러한 위치를 토큰 인덱스로 매핑하려면 offset 매핑을 사용해야 합니다. \nreturn_offsets_mapping=True를 사용하여 tokenizer로부터 이러한 offset 매핑을 얻을 수 있습니다.\n\n*즉, answer span이 context에 같이 존재하지 않으면 (0,0)의 index를 반환한다.\n또한, 같은 context임을 알려주기 위해 offset을 도입.\n'

In [15]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True, # 이전 예제에서 추가된 요소
)
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [16]:
"""
여기서 보듯이 우리는 일반적인 입력 ID, 토큰 유형 ID, 어텐션 마스크를 얻을 수 있으며, 
필요한 offset 매핑과 추가적인 overflow_to_sample_mapping 키를 얻습니다. 
해당 값은 여러 개의 텍스트를 동시에 토큰화할 때 유용하며 (tokenizer가 Rust로 백엔드 처리를 지원하기 때문에 이렇게 하는 것이 좋습니다), 
각 피처를 원본에서 유래한 예제에 매핑합니다. 
이 예에서는 하나의 예제만 토큰화했기 때문에 0의 목록을 얻습니다.
"""

'\n여기서 보듯이 우리는 일반적인 입력 ID, 토큰 유형 ID, 어텐션 마스크를 얻을 수 있으며, \n필요한 offset 매핑과 추가적인 overflow_to_sample_mapping 키를 얻습니다. \n해당 값은 여러 개의 텍스트를 동시에 토큰화할 때 유용하며 (tokenizer가 Rust로 백엔드 처리를 지원하기 때문에 이렇게 하는 것이 좋습니다), \n각 피처를 원본에서 유래한 예제에 매핑합니다. \n이 예에서는 하나의 예제만 토큰화했기 때문에 0의 목록을 얻습니다.\n'

In [17]:
"""같은 context임을 알려준다"""
inputs["overflow_to_sample_mapping"]

[0, 0, 0, 0]

In [18]:
inputs = tokenizer(
    raw_datasets["train"][2:6]["question"],
    raw_datasets["train"][2:6]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 4 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 4 examples gave 17 features.
Here is where each comes from: [0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3].


In [19]:
"""
*이전에 언급한대로 레이블은 다음과 같습니다.
    (0, 0) : 답이 해당 컨텍스트 부분에 없을 경우
    (start_position, end_position) : 답이 해당 컨텍스트 부분에 있을 경우, 
                                     start_position은 답의 시작 지점을 나타내는 토큰의 인덱스이며, 
                                     end_position은 답의 끝 지점을 나타내는 토큰의 인덱스입니다.

*이 상황을 판단하고 필요한 경우 토큰의 위치를 찾기 위해, 먼저 입력 ID에서 컨텍스트의 시작과 끝을 나타내는 토큰 인덱스를 찾습니다. 
토큰 유형 ID를 사용할 수 있지만, 이 모든 모델에서 필요하지 않을 수 있으므로 (예를 들어 DistilBERT는 필요로하지 않음), 
대신 tokenizer가 반환하는 BatchEncoding의 sequence_ids() 메서드를 사용합니다.

*한 번 이러한 토큰 인덱스를 얻으면 해당 오프셋을 확인하며, 
이것은 원본 컨텍스트 내의 문자 범위를 나타내는 두 정수로 구성된 튜플입니다. 
이로써 해당 기능의 컨텍스트 부분이 답변 이후에 시작하거나 답변 시작 이전에 끝날 경우 (0, 0) 레이블이라고 판단할 수 있습니다. 
그렇지 않은 경우, 답의 첫 번째 토큰과 마지막 토큰을 찾기 위해 루프를 돕니다.
"""

'\n*이전에 언급한대로 레이블은 다음과 같습니다.\n    (0, 0) : 답이 해당 컨텍스트 부분에 없을 경우\n    (start_position, end_position) : 답이 해당 컨텍스트 부분에 있을 경우, \n                                     start_position은 답의 시작 지점을 나타내는 토큰의 인덱스이며, \n                                     end_position은 답의 끝 지점을 나타내는 토큰의 인덱스입니다.\n\n*이 상황을 판단하고 필요한 경우 토큰의 위치를 찾기 위해, 먼저 입력 ID에서 컨텍스트의 시작과 끝을 나타내는 토큰 인덱스를 찾습니다. \n토큰 유형 ID를 사용할 수 있지만, 이 모든 모델에서 필요하지 않을 수 있으므로 (예를 들어 DistilBERT는 필요로하지 않음), \n대신 tokenizer가 반환하는 BatchEncoding의 sequence_ids() 메서드를 사용합니다.\n\n*한 번 이러한 토큰 인덱스를 얻으면 해당 오프셋을 확인하며, \n이것은 원본 컨텍스트 내의 문자 범위를 나타내는 두 정수로 구성된 튜플입니다. \n이로써 해당 기능의 컨텍스트 부분이 답변 이후에 시작하거나 답변 시작 이전에 끝날 경우 (0, 0) 레이블이라고 판단할 수 있습니다. \n그렇지 않은 경우, 답의 첫 번째 토큰과 마지막 토큰을 찾기 위해 루프를 돕니다.\n'

In [20]:
answers = raw_datasets["train"][2:6]["answers"]
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions

([81, 49, 17, 0, 0, 57, 19, 33, 0, 0, 0, 63, 27, 0, 0, 0, 0],
 [83, 51, 19, 0, 0, 63, 25, 39, 0, 0, 0, 64, 28, 0, 0, 0, 0])

In [21]:
"""
우리의 접근 방법이 올바른지 확인하기 위해 몇 가지 결과를 살펴보겠습니다. 
첫 번째 feature에서 우리는 (83, 85) 레이블을 찾아, 83에서 85까지(포함) 토큰의 디코드된 범위와 이론적인 답변을 비교해 보겠습니다.
"""

'\n우리의 접근 방법이 올바른지 확인하기 위해 몇 가지 결과를 살펴보겠습니다. \n첫 번째 feature에서 우리는 (83, 85) 레이블을 찾아, 83에서 85까지(포함) 토큰의 디코드된 범위와 이론적인 답변을 비교해 보겠습니다.\n'

In [22]:
idx = 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: the Main Building, labels give: the main building


In [23]:
"""
이건 일치하는 경우입니다.
이제 인덱스 4를 확인해보면 거기서는 레이블을 (0, 0)으로 설정했기 때문에, 해당 feature의 컨텍스트 청크에 답변이 포함되어 있지 않음을 의미합니다.
"""

'\n이건 일치하는 경우입니다.\n이제 인덱스 4를 확인해보면 거기서는 레이블을 (0, 0)으로 설정했기 때문에, 해당 feature의 컨텍스트 청크에 답변이 포함되어 있지 않음을 의미합니다.\n'

In [24]:
idx = 4
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

decoded_example = tokenizer.decode(inputs["input_ids"][idx])
print(f"Theoretical answer: {answer}, decoded example: {decoded_example}")

Theoretical answer: a Marian place of prayer and reflection, decoded example: [CLS] what is the grotto at notre dame? [SEP] architecturally, the school has a catholic character. atop the main building's gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it, is a copper statue of christ with arms upraised with the legend " venite ad me omnes ". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian place of [SEP]


In [25]:
"""
네, 이 특정한 feature의 컨텍스트 안에 "Bernadette Soubirous"라는 정답이 포함되어 있지 않으므로 이 경우 레이블이 (0, 0)으로 설정됩니다. 
이것은 예상된 동작과 일치합니다. 이 feature의 컨텍스트 내에 정답이 없기 때문에 레이블이 (0, 0)으로 설정된 것입니다.
"""

'\n네, 이 특정한 feature의 컨텍스트 안에 "Bernadette Soubirous"라는 정답이 포함되어 있지 않으므로 이 경우 레이블이 (0, 0)으로 설정됩니다. \n이것은 예상된 동작과 일치합니다. 이 feature의 컨텍스트 내에 정답이 없기 때문에 레이블이 (0, 0)으로 설정된 것입니다.\n'

In [26]:
"""
전체 교육 데이터셋에 적용할 함수를 작성할 차례입니다. 
대부분의 컨텍스트가 길기 때문에 (그에 해당하는 샘플이 여러 feature로 분할될 것입니다) 모든 feature를 설정한 최대 길이로 패딩합니다. 
동적 패딩을 적용하는 것에는 실제로 이득이 없기 때문입니다.
"""

'\n전체 교육 데이터셋에 적용할 함수를 작성할 차례입니다. \n대부분의 컨텍스트가 길기 때문에 (그에 해당하는 샘플이 여러 feature로 분할될 것입니다) 모든 feature를 설정한 최대 길이로 패딩합니다. \n동적 패딩을 적용하는 것에는 실제로 이득이 없기 때문입니다.\n'

In [27]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [28]:
"""
최대 길이 및 슬라이딩 창의 길이를 결정하는 두 개의 상수를 정의하고 토큰화하기 전에 약간의 정리를 추가로 수행했다는 점을 유의하세요. 
SQuAD 데이터셋의 일부 질문은 아무런 내용을 추가하지 않지만 (RoBERTa와 같은 모델을 사용할 때 토큰화될 때 공간을 차지함) 
시작과 끝에 추가 공백이 있으므로 이러한 추가 공백을 제거했습니다.

이 기능을 전체 교육 세트에 적용하려면 Dataset.map() 메서드를 batched=True 플래그와 함께 사용해야 합니다. 
이것은 데이터셋의 길이를 변경하는 작업이기 때문에 여기에서 필요한 작업입니다. 하나의 예제가 여러 교육 피처를 생성할 수 있기 때문입니다.
"""

'\n최대 길이 및 슬라이딩 창의 길이를 결정하는 두 개의 상수를 정의하고 토큰화하기 전에 약간의 정리를 추가로 수행했다는 점을 유의하세요. \nSQuAD 데이터셋의 일부 질문은 아무런 내용을 추가하지 않지만 (RoBERTa와 같은 모델을 사용할 때 토큰화될 때 공간을 차지함) \n시작과 끝에 추가 공백이 있으므로 이러한 추가 공백을 제거했습니다.\n\n이 기능을 전체 교육 세트에 적용하려면 Dataset.map() 메서드를 batched=True 플래그와 함께 사용해야 합니다. \n이것은 데이터셋의 길이를 변경하는 작업이기 때문에 여기에서 필요한 작업입니다. 하나의 예제가 여러 교육 피처를 생성할 수 있기 때문입니다.\n'

In [29]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset)

(87599, 88524)

In [30]:
"""
검증 데이터를 전처리하는 작업은 라벨을 생성할 필요가 없기 때문에 약간 더 쉬울 것입니다 
(검증 손실을 계산하려는 경우에만 라벨을 생성해야 하지만 그 숫자는 모델의 성능을 이해하는 데 도움이 되지 않을 것입니다). 
실제 즐거움은 모델의 예측을 원래 문맥의 범위로 해석하는 것입니다. 
이를 위해 offset 매핑과 각 생성된 피처를 원본 예제와 일치시키는 방법이 필요합니다. 
원본 데이터셋에 ID 열이 있으므로 해당 ID를 사용할 것입니다.

여기에 추가할 것은 offset 매핑을 약간 정리하는 작업뿐입니다. 
이러한 매핑에는 질문과 문맥에 대한 오프셋이 포함되지만 후속 처리 단계에서는 입력 ID의 어떤 부분이 문맥에 해당하고 어떤 부분이 질문인지 알 방법이 없습니다 
(우리가 사용한 sequence_ids() 메서드는 토큰화기의 출력에만 사용할 수 있습니다). 따라서 질문에 해당하는 오프셋을 None으로 설정할 것입니다.
"""

'\n검증 데이터를 전처리하는 작업은 라벨을 생성할 필요가 없기 때문에 약간 더 쉬울 것입니다 \n(검증 손실을 계산하려는 경우에만 라벨을 생성해야 하지만 그 숫자는 모델의 성능을 이해하는 데 도움이 되지 않을 것입니다). \n실제 즐거움은 모델의 예측을 원래 문맥의 범위로 해석하는 것입니다. \n이를 위해 offset 매핑과 각 생성된 피처를 원본 예제와 일치시키는 방법이 필요합니다. \n원본 데이터셋에 ID 열이 있으므로 해당 ID를 사용할 것입니다.\n\n여기에 추가할 것은 offset 매핑을 약간 정리하는 작업뿐입니다. \n이러한 매핑에는 질문과 문맥에 대한 오프셋이 포함되지만 후속 처리 단계에서는 입력 ID의 어떤 부분이 문맥에 해당하고 어떤 부분이 질문인지 알 방법이 없습니다 \n(우리가 사용한 sequence_ids() 메서드는 토큰화기의 출력에만 사용할 수 있습니다). 따라서 질문에 해당하는 오프셋을 None으로 설정할 것입니다.\n'

In [31]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [32]:
"""이러한 함수를 이전과 같이 검증 데이터셋 전체에 적용할 수 있습니다."""

'이러한 함수를 이전과 같이 검증 데이터셋 전체에 적용할 수 있습니다.'

In [33]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

(10570, 10784)

In [34]:
"""모든 데이터를 전처리했으므로 이제 훈련에 진입할 수 있습니다."""

'모든 데이터를 전처리했으므로 이제 훈련에 진입할 수 있습니다.'

## Fine-tuning the model with the Trainer API

In [35]:
"""Small validation dataset으로 evaluate"""

'Small validation dataset으로 evaluate'

In [36]:
"""SQuAD 용으로 학습된 distilbert로 evaluate 할거여서 tokenizer 불러오기"""
small_eval_set = raw_datasets["validation"].select(range(100))
trained_checkpoint = "distilbert-base-cased-distilled-squad"

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)

In [37]:
#model_checkpoint = "distilbert-base-uncased"
"""나중에 fine-tuning은 SQuAD로 학습된 거로 안할꺼니까 다시 바꿔주기"""
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [38]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}

"""SQuAD로 이미 학습된 distilbert model 불러오기"""
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(
    device
)

with torch.no_grad():
    outputs = trained_model(**batch)

/home/qkrwnstj/anaconda3/envs/distilbert/lib/python3.9/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [39]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [40]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)

In [41]:
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

for example in small_eval_set:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = eval_set["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})

In [42]:
"""Huging Face에서 제공하는 library로 evaluate"""
import evaluate

metric = evaluate.load("squad")

In [43]:
theoretical_answers = [
    {"id": ex["id"], "answers": ex["answers"]} for ex in small_eval_set
]

In [44]:
print(predicted_answers[0])
print(theoretical_answers[0])

{'id': '56be4db0acb8001400a502ec', 'prediction_text': 'Denver Broncos'}
{'id': '56be4db0acb8001400a502ec', 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'], 'answer_start': [177, 177, 177]}}


In [45]:
"""SQuAD dataset에 대해서 잘 학습된 model의 evaluate score"""
metric.compute(predictions=predicted_answers, references=theoretical_answers)

{'exact_match': 83.0, 'f1': 88.25000000000004}

<img src='./distilbert_result.png'></img>
</br>
DistilBERT paper에서 작성한 결과 table이다. </br>SQuAD 1.1 dataset에 대해서, <strong>79.1/86.9(Exact Match/F1 score)</strong>를 달성했다.

In [46]:
"""Hugging Face의 evaluate method 구현"""
from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [47]:
compute_metrics(start_logits, end_logits, eval_set, small_eval_set)

  0%|          | 0/100 [00:00<?, ?it/s]

{'exact_match': 83.0, 'f1': 88.25000000000004}

## Fine-tuning the model
이전 section에선, 이미 SQuAD dataset으로 학습된 distilbert를 이용해서 결과를 미리 확인해봤다. </br>
이번 section에서는 직접 fine-tuning을 해보는 section이다.

In [48]:
print(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

distilbert-base-uncased


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
from huggingface_hub import notebook_login

notebook_login()

In [50]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [51]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="distilbert-finetuned-squad-test2",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    #fp16=True,
    push_to_hub=True,
)

In [52]:
"""
다음의 ERROR가 발생해서 추가
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
"""
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [62]:
os.environ["WANDB_START_METHOD"]='thread'

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f528d8bce20, raw_cell="os.environ["WANDB_START_METHOD"]='thread'" store_history=True silent=False shell_futures=True cell_id=1f06dd62-9bad-4483-838a-1fd2bdaabdc1>,),kwargs {}:


TypeError: _resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f528d8bcbb0, execution_count=62 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f528d8bce20, raw_cell="os.environ["WANDB_START_METHOD"]='thread'" store_history=True silent=False shell_futures=True cell_id=1f06dd62-9bad-4483-838a-1fd2bdaabdc1> result=None>,),kwargs {}:


TypeError: _pause_backend() takes 1 positional argument but 2 were given

In [53]:
"""
FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. 
Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
"""
from transformers import AdamW

In [54]:
#command창에서 실행
#wandb login --relogin

In [55]:
os.environ["NCCL_DEBUG"] = "INFO"

In [56]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: qkrwnstj300 (qkrwnstj). Use `wandb login --relogin` to force relogin


user-WS-C621E-SAGE-Series:3239038:3239038 [0] NCCL INFO cudaDriverVersion 11040
user-WS-C621E-SAGE-Series:3239038:3239038 [0] NCCL INFO Bootstrap : Using enp6s0:210.94.179.16<0>
user-WS-C621E-SAGE-Series:3239038:3239038 [0] NCCL INFO NET/Plugin : No plugin found (libnccl-net.so), using internal implementation
NCCL version 2.14.3+cuda11.8
user-WS-C621E-SAGE-Series:3239038:3239462 [0] NCCL INFO Failed to open libibverbs.so[.1]
user-WS-C621E-SAGE-Series:3239038:3239462 [0] NCCL INFO NET/Socket : Using [0]enp6s0:210.94.179.16<0>
user-WS-C621E-SAGE-Series:3239038:3239462 [0] NCCL INFO Using network Socket
user-WS-C621E-SAGE-Series:3239038:3239464 [2] NCCL INFO Using network Socket
user-WS-C621E-SAGE-Series:3239038:3239465 [3] NCCL INFO Using network Socket
user-WS-C621E-SAGE-Series:3239038:3239463 [1] NCCL INFO Using network Socket

user-WS-C621E-SAGE-Series:3239038:3239464 [2] misc/nvmlwrap.cc:98 NCCL WARN nvmlInit_v2() failed: Driver/library version mismatch
user-WS-C621E-SAGE-Series:3239

/home/qkrwnstj/anaconda3/envs/distilbert/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,1.493200,No log
2,1.119000,No log
3,0.981600,No log


/home/qkrwnstj/anaconda3/envs/distilbert/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/qkrwnstj/anaconda3/envs/distilbert/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/qkrwnstj/anaconda3/envs/distilbert/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/qkrwnstj/anaconda3/envs/distilbert/lib/python3.9/site-packages/torch/nn/parallel/_fun

TrainOutput(global_step=4152, training_loss=1.3141003316537494, metrics={'train_runtime': 1308.8918, 'train_samples_per_second': 202.898, 'train_steps_per_second': 3.172, 'total_flos': 2.602335381127373e+16, 'train_loss': 1.3141003316537494, 'epoch': 3.0})

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f50d47bd130, execution_count=56 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f51e18d3af0, raw_cell="from transformers import Trainer

trainer = Traine.." store_history=True silent=False shell_futures=True cell_id=ed0b4f38-b6c1-4c28-a0a9-d967134c6670> result=TrainOutput(global_step=4152, training_loss=1.3141003316537494, metrics={'train_runtime': 1308.8918, 'train_samples_per_second': 202.898, 'train_steps_per_second': 3.172, 'total_flos': 2.602335381127373e+16, 'train_loss': 1.3141003316537494, 'epoch': 3.0})>,),kwargs {}:


TypeError: _pause_backend() takes 1 positional argument but 2 were given

In [ ]:
"""
모델을 평가하기 위해 학습이 완료되면 predict() 메서드를 사용하여 모델의 예측을 가져올 수 있습니다. 
이 메서드는 튜플을 반환하며, 첫 번째 요소는 모델의 예측값입니다 (여기서는 시작 및 종료 로짓의 쌍). 이 값을 compute_metrics() 함수로 보내서 모델을 평가합니다.
"""

In [57]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"])

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f528e96d3d0, raw_cell="predictions, _, _ = trainer.predict(validation_dat.." store_history=True silent=False shell_futures=True cell_id=39ba9afa-f3fe-492d-9688-521d4328ce8d>,),kwargs {}:


TypeError: _resume_backend() takes 1 positional argument but 2 were given

  0%|          | 0/10570 [00:00<?, ?it/s]

{'exact_match': 74.8155156102176, 'f1': 83.53339500367704}

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f528e96d9d0, execution_count=57 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f528e96d3d0, raw_cell="predictions, _, _ = trainer.predict(validation_dat.." store_history=True silent=False shell_futures=True cell_id=39ba9afa-f3fe-492d-9688-521d4328ce8d> result={'exact_match': 74.8155156102176, 'f1': 83.53339500367704}>,),kwargs {}:


TypeError: _pause_backend() takes 1 positional argument but 2 were given

In [58]:
trainer.push_to_hub(commit_message="Training complete")

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f511c1c8bb0, raw_cell="trainer.push_to_hub(commit_message="Training compl.." store_history=True silent=False shell_futures=True cell_id=9991765e-d327-4ee7-9d96-455a02187576>,),kwargs {}:


TypeError: _resume_backend() takes 1 positional argument but 2 were given

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

'https://huggingface.co/qkrwnstj/distilbert-finetuned-squad-test2/tree/main/'

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f528d8f2cd0, execution_count=58 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f511c1c8bb0, raw_cell="trainer.push_to_hub(commit_message="Training compl.." store_history=True silent=False shell_futures=True cell_id=9991765e-d327-4ee7-9d96-455a02187576> result='https://huggingface.co/qkrwnstj/distilbert-finetuned-squad-test2/tree/main/'>,),kwargs {}:


TypeError: _pause_backend() takes 1 positional argument but 2 were given

# Custom Fine-Tuning

In [ ]:
"""
데이터 형식을 torch로 설정하고 모델에서 사용되지 않는 검증 데이터셋의 열을 제거한다.
그 후, transformers에서 제공하는 default_data_collator를 collate_fn으로 사용하여 훈련 데이터셋을 섞고 
검증 데이터셋은 섞지 않습니다.
"""

In [59]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=32,
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=32
)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f528d1fa850, raw_cell="from torch.utils.data import DataLoader
from trans.." store_history=True silent=False shell_futures=True cell_id=b9ae098a-f400-4e28-bb23-8f9aa49bb687>,),kwargs {}:


TypeError: _resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f528d1fa6d0, execution_count=59 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f528d1fa850, raw_cell="from torch.utils.data import DataLoader
from trans.." store_history=True silent=False shell_futures=True cell_id=b9ae098a-f400-4e28-bb23-8f9aa49bb687> result=None>,),kwargs {}:


TypeError: _pause_backend() takes 1 positional argument but 2 were given

In [60]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f528d916580, raw_cell="model = AutoModelForQuestionAnswering.from_pretrai.." store_history=True silent=False shell_futures=True cell_id=489ea90c-2bb7-467e-af55-a3417c9afe89>,),kwargs {}:


TypeError: _resume_backend() takes 1 positional argument but 2 were given

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f528d916a30, execution_count=60 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f528d916580, raw_cell="model = AutoModelForQuestionAnswering.from_pretrai.." store_history=True silent=False shell_futures=True cell_id=489ea90c-2bb7-467e-af55-a3417c9afe89> result=None>,),kwargs {}:


TypeError: _pause_backend() takes 1 positional argument but 2 were given

In [63]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f528d8f2d90, raw_cell="from torch.optim import AdamW

optimizer = AdamW(m.." store_history=True silent=False shell_futures=True cell_id=928133b3-9075-4258-bad3-0e651ab96a0c>,),kwargs {}:


TypeError: _resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f528d8f2df0, execution_count=63 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f528d8f2d90, raw_cell="from torch.optim import AdamW

optimizer = AdamW(m.." store_history=True silent=False shell_futures=True cell_id=928133b3-9075-4258-bad3-0e651ab96a0c> result=None>,),kwargs {}:


TypeError: _pause_backend() takes 1 positional argument but 2 were given

In [65]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f528d1fa820, raw_cell="from accelerate import Accelerator

accelerator = .." store_history=True silent=False shell_futures=True cell_id=8638a054-7242-43ee-8729-0146dc138a57>,),kwargs {}:


TypeError: _resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f528d1fa850, execution_count=65 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f528d1fa820, raw_cell="from accelerate import Accelerator

accelerator = .." store_history=True silent=False shell_futures=True cell_id=8638a054-7242-43ee-8729-0146dc138a57> result=None>,),kwargs {}:


TypeError: _pause_backend() takes 1 positional argument but 2 were given

In [66]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f528d6fcbe0, raw_cell="from transformers import get_scheduler

num_train_.." store_history=True silent=False shell_futures=True cell_id=9d48503b-3461-4ee8-9aad-aee44e2b6850>,),kwargs {}:


TypeError: _resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f528d6fc610, execution_count=66 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f528d6fcbe0, raw_cell="from transformers import get_scheduler

num_train_.." store_history=True silent=False shell_futures=True cell_id=9d48503b-3461-4ee8-9aad-aee44e2b6850> result=None>,),kwargs {}:


TypeError: _pause_backend() takes 1 positional argument but 2 were given

In [67]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-finetuned-squad-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f528d6fc5e0, raw_cell="from huggingface_hub import Repository, get_full_r.." store_history=True silent=False shell_futures=True cell_id=8087666d-93ad-4433-b911-ce4708f3238c>,),kwargs {}:


TypeError: _resume_backend() takes 1 positional argument but 2 were given

'qkrwnstj/bert-finetuned-squad-accelerate'

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f528d6fcc10, execution_count=67 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f528d6fc5e0, raw_cell="from huggingface_hub import Repository, get_full_r.." store_history=True silent=False shell_futures=True cell_id=8087666d-93ad-4433-b911-ce4708f3238c> result='qkrwnstj/bert-finetuned-squad-accelerate'>,),kwargs {}:


TypeError: _pause_backend() takes 1 positional argument but 2 were given

In [1]:
output_dir = "bert-finetuned-squad-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

NameError: name 'Repository' is not defined

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    start_logits = []
    end_logits = []
    accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets["validation"]
    )
    print(f"epoch {epoch}:", metrics)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f50d41d2190>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f528d6fc8b0, raw_cell="from tqdm.auto import tqdm
import torch

progress_.." store_history=True silent=False shell_futures=True cell_id=af4a3bd4-40ae-415c-af75-530a51522e57>,),kwargs {}:


TypeError: _resume_backend() takes 1 positional argument but 2 were given

  0%|          | 0/8301 [00:00<?, ?it/s]

In [ ]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "qkrwnstj/bert-finetuned-squad"
question_answerer = pipeline("question-answering", model=model_checkpoint)

context = """
🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question = "Which deep learning libraries back 🤗 Transformers?"
question_answerer(question=question, context=context)